In [2]:
import pandas as pd
import requests
import json
from pymongo import MongoClient
from pymongo import GEOSPHERE
import seaborn as sns
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from dotenv import load_dotenv
import os
from functools import reduce
import operator
load_dotenv()

True

In [3]:
### FOURSQUARE
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [4]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")


In [5]:
companies = db.get_collection("companies")
gaming_proj = {"_id": 0, "name": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
gaming_companies = list(companies.find({"number_of_employees": {"$gt": 50}, "total_money_raised": {"$gte": "$1M"}, "offices.zip_code": {"$exists": True}, "offices.city": {"$exists": True}}, gaming_proj))
gaming_companies

[{'name': 'Digg',
  'number_of_employees': 60,
  'offices': [{'latitude': 37.764726, 'longitude': -122.394523}]},
 {'name': 'Facebook',
  'number_of_employees': 5299,
  'offices': [{'latitude': 37.41605, 'longitude': -122.151801},
   {'latitude': 53.344104, 'longitude': -6.267494},
   {'latitude': 40.7557162, 'longitude': -73.9792469}]},
 {'name': 'eBay',
  'number_of_employees': 15000,
  'offices': [{'latitude': 37.295005, 'longitude': -121.930035}]},
 {'name': 'Cisco',
  'number_of_employees': 63000,
  'offices': [{'latitude': 37.408802, 'longitude': -121.95377}]},
 {'name': 'Yahoo!',
  'number_of_employees': 13600,
  'offices': [{'latitude': 37.418531, 'longitude': -122.025485}]},
 {'name': 'Powerset',
  'number_of_employees': 60,
  'offices': [{'latitude': 37.778613, 'longitude': -122.395289}]},
 {'name': 'AddThis',
  'number_of_employees': 120,
  'offices': [{'latitude': 38.926172, 'longitude': -77.245195},
   {'latitude': 40.724604, 'longitude': -73.996876},
   {'latitude': 34.02

In [8]:
gaming_companies = db.get_collection("gaming_companies")
ny_companies = list(gaming_companies.find())
ny_companies
new_york = pd.DataFrame(ny_companies)
latitude = [lat["offices"][0]["latitude"] for lat in ny_companies]
longitude = [long["offices"][0]["longitude"] for long in ny_companies]
zip_code = [code["offices"][0]["zip_code"] for code in ny_companies]
address = [point["offices"][0]["address1"] for point in ny_companies]

#De aquí añadimos las listas que nos interesen al DataFrame de la ciudad de San Francisco.
new_york["latitude"] = latitude
new_york["longitude"] = longitude
new_york["zip_code"] = zip_code
new_york["address"] = address

new_york = new_york[new_york["latitude"].notna()]
new_york = new_york[new_york["longitude"].notna()]

new_york

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,milestones,video_embeds,screenshots,external_links,partners,ipo,latitude,longitude,zip_code,address
0,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],NaN,37.764726,-122.394523,94107,135 Mississippi St
1,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,"[{'id': 1, 'description': 'Survives iPhone 3G ...","[{'embed_code': '<iframe width=""430"" height=""2...","[{'available_sizes': [[[150, 74], 'assets/imag...",[{'external_url': 'http://sec.gov/cgi-bin/brow...,[],"{'valuation_amount': 18100000000, 'valuation_c...",37.776805,-122.416924,94103,1355 Market St.
2,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,"[{'id': 3103, 'description': 'Spin-out from Eb...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.facebook.com/Stu...,[],NaN,37.775196,-122.419204,,
3,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,"[{'id': 4960, 'description': 'Scribd launches ...","[{'embed_code': '<script src=""http://i.cdn.tur...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.online-tech-tips...,[],NaN,37.789634,-122.404052,94107,539 Bryant Street
4,52cdef7c4bab8bd675297da4,Powerset,powerset,http://www.crunchbase.com/company/powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60.0,...,[],"[{'embed_code': '<object width=""400"" height=""3...",[],[{'external_url': 'http://en.wikipedia.org/wik...,[],NaN,37.778613,-122.395289,94107,475 Brannan St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,52cdef7f4bab8bd67529c4c8,Danoo,danoo,http://www.crunchbase.com/company/danoo,http://danoo.com,,,,advertising,NaN,...,"[{'id': 15727, 'description': 'Renamed RMG Net...",[],[],[],[],NaN,37.788852,-122.400863,94111,1000 Sansome St.
897,52cdef7f4bab8bd67529c4e1,UCWeb,ucweb,http://www.crunchbase.com/company/ucweb,http://www.ucweb.com,,,UCBrowser,software,1700.0,...,"[{'id': 35730, 'description': 'Opened U.S. off...","[{'embed_code': '<iframe width=""560"" height=""3...","[{'available_sizes': [[[150, 59], 'assets/imag...",[{'external_url': 'http://thenextweb.com/insid...,[],NaN,23.133754,113.385331,510627,"GRG Plaza,No. 163,Pingyun Rd.,Huangpu West Ave..."
898,52cdef7f4bab8bd67529c565,Grassroots,grassroots,http://www.crunchbase.com/company/grassroots,http://www.grassroots.com,,,,public_relations,NaN,...,[],[],[],[],[],NaN,38.901294,-77.032034,20006,1625 Eye Street NW
899,52cdef7f4bab8bd67529c5e3,DJ Nitrogen,dj-nitrogen,http://www.crunchbase.com/company/dj-nitrogen,http://www.djnitrogen.com,http://news.djnitrogen.com/,,djnitrogen,games_video,8.0,...,[],[],"[{'available_sizes': [[[150, 75], 'assets/imag...",[{'external_url': 'http://www.cnet.com/8301-19...,[],NaN,37.784174,-122.449408,94118,11 Lupine Avenue


In [10]:
#Geoquery inicial

ny = {'type': 'Point', 'coordinates': [40.6643, -73.9385]}
url_query = 'https://api.foursquare.com/v2/venues/explore'

parametros = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "basketball", 
    "limit": 100    
}

In [12]:
resp = requests.get(url= url_query, params = parametros).json()
data = resp.get("response").get("groups")[0].get("items")

In [13]:
#Vamos a sacar el primer DF
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lng"]
mapa_longitud = ["venue", "location", "lat"]

def getFromDict(diccionario,mapa):
    
    return reduce(operator.getitem,mapa,diccionario)

basket_list = []
for dic in data:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    basket_list.append(paralista)
    

documents = []
for diccionario in basket_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
basket_venues = pd.DataFrame(documents)
basket_venues["place"] = "basketball court"
basket_venues.head()

,name,location,place
0,Brower Park,"{'type': 'Point', 'coordinates': [-73.94349347...",basketball court
1,St. John's Park,"{'type': 'Point', 'coordinates': [-73.93476963...",basketball court
2,Prospect Park Parade Ground,"{'type': 'Point', 'coordinates': [-73.96828651...",basketball court
3,St. Andrews Playground,"{'type': 'Point', 'coordinates': [-73.94155323...",basketball court
4,Nostrand Playground,"{'type': 'Point', 'coordinates': [-73.94805697...",basketball court


In [14]:
basket_venues.to_json("basket_venues_ny.json", orient="records")

In [15]:
#Sacamos vegan_venues.

parametros_1 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "vegan", 
    "limit": 100    
}

In [16]:
resp_1 = requests.get(url= url_query, params = parametros_1).json()
data_1 = resp_1.get("response").get("groups")[0].get("items")
data_1

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4d4480681911a09332ded8d8',
   'name': 'Italfari',
   'contact': {},
   'location': {'address': '313 Utica Ave',
    'lat': 40.67033692548684,
    'lng': -73.93101360938604,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.67033692548684,
      'lng': -73.93101360938604}],
    'distance': 922,
    'postalCode': '11213',
    'cc': 'US',
    'city': 'Brooklyn',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['313 Utica Ave',
     'Brooklyn, NY 11213',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1d3941735',
     'name': 'Vegetarian / Vegan Restaurant',
     'pluralName': 'Vegetarian / Vegan Restaurants',
     'shortName': 'Vegetarian / Vegan',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
      'suffix': '.png'},
     'primary': T

In [17]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

vegan_list = []
for dic in data_1:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    vegan_list.append(paralista)
    

documents = []
for diccionario in vegan_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
vegan_venues = pd.DataFrame(documents)
vegan_venues["place"] = "vegan"
vegan_venues.head()

,name,location,place
0,Italfari,"{'type': 'Point', 'coordinates': [-73.93101360...",vegan
1,Punch Line Juices,"{'type': 'Point', 'coordinates': [-73.95022459...",vegan
2,Veggies Natural Juice Bar,"{'type': 'Point', 'coordinates': [-73.95749476...",vegan
3,Natural Blend,"{'type': 'Point', 'coordinates': [-73.96292344...",vegan
4,Ital Kitchen,"{'type': 'Point', 'coordinates': [-73.958035, ...",vegan


In [42]:
vegan_venues.to_json("vegan_venues_ny.json", orient="records")

In [21]:
parametros_2 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "starbucks", 
    "limit": 100    
}

In [22]:
resp_2 = requests.get(url= url_query, params = parametros_2).json()
data_2 = resp_2.get("response").get("groups")[0].get("items")
data_2

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '54173fc4498eda4d0bcc2be8',
   'name': 'Starbucks',
   'contact': {},
   'location': {'address': '341 Eastern Pkwy',
    'crossStreet': 'at Franklin Ave',
    'lat': 40.6711284292753,
    'lng': -73.95778421058986,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.6711284292753,
      'lng': -73.95778421058986},
     {'label': 'entrance', 'lat': 40.670945, 'lng': -73.957515}],
    'distance': 1796,
    'postalCode': '11238',
    'cc': 'US',
    'city': 'Brooklyn',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['341 Eastern Pkwy (at Franklin Ave)',
     'Brooklyn, NY 11238',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'shortName': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi

In [23]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

starbucks_list = []
for dic in data_2:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    starbucks_list.append(paralista)
    

documents = []
for diccionario in starbucks_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
starbucks_venues = pd.DataFrame(documents)
starbucks_venues["place"] = "starbucks"
starbucks_venues.head()

,name,location,place
0,Starbucks,"{'type': 'Point', 'coordinates': [-73.95778421...",starbucks
1,Starbucks,"{'type': 'Point', 'coordinates': [-73.96267818...",starbucks
2,Starbucks,"{'type': 'Point', 'coordinates': [-73.97657657...",starbucks
3,Starbucks,"{'type': 'Point', 'coordinates': [-73.975945, ...",starbucks
4,Starbucks,"{'type': 'Point', 'coordinates': [-73.98982298...",starbucks


In [24]:
starbucks_venues.to_json("starbucks_venues_ny.json", orient="records")

In [25]:
parametros_3 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "school", 
    "limit": 100    
}

In [26]:
resp_3 = requests.get(url= url_query, params = parametros_3).json()
data_3 = resp_3.get("response").get("groups")[0].get("items")
data_3

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4cc3299c914137046fd1b955',
   'name': 'Old Boys High Field',
   'contact': {},
   'location': {'crossStreet': 'Rutland rd. & Troy ave.',
    'lat': 40.65866658523819,
    'lng': -73.93842673009168,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.65866658523819,
      'lng': -73.93842673009168}],
    'distance': 627,
    'cc': 'US',
    'city': 'Brooklyn',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['Brooklyn, NY', 'United States']},
   'categories': [{'id': '4bf58dd8d48988d15f941735',
     'name': 'Field',
     'pluralName': 'Fields',
     'shortName': 'Field',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/field_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'checkins

In [27]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

school_list = []
for dic in data_3:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    school_list.append(paralista)
    

documents = []
for diccionario in school_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
school_venues = pd.DataFrame(documents)
school_venues["place"] = "school"
school_venues.head()

,name,location,place
0,Old Boys High Field,"{'type': 'Point', 'coordinates': [-73.93842673...",school
1,PS 91,"{'type': 'Point', 'coordinates': [-73.94019275...",school
2,Batsheva Academy,"{'type': 'Point', 'coordinates': [-73.93719899...",school
3,Lubavitcher Yeshivah,"{'type': 'Point', 'coordinates': [-73.93955458...",school
4,Ps 221,"{'type': 'Point', 'coordinates': [-73.93595054...",school


In [28]:
school_venues.to_json("school_venues_ny.json", orient="records")

In [30]:
parametros_4 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "airport", 
    "limit": 100    
}

In [31]:
resp_4 = requests.get(url= url_query, params = parametros_4).json()
data_4 = resp_4.get("response").get("groups")[0].get("items")
data_4

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '53c3d15b498e81100fd43669',
   'name': 'JFK Terminal 7 British Airways',
   'contact': {},
   'location': {'lat': 40.660392811487704,
    'lng': -73.9098407849627,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.660392811487704,
      'lng': -73.9098407849627}],
    'distance': 2458,
    'cc': 'US',
    'city': 'Queens',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['Queens, NY', 'United States']},
   'categories': [{'id': '4bf58dd8d48988d1eb931735',
     'name': 'Airport Terminal',
     'pluralName': 'Airport Terminals',
     'shortName': 'Terminal',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_gate_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'checkinsCount': 0,
   

In [32]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

airport_list = []
for dic in data_4:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    airport_list.append(paralista)
    

documents = []
for diccionario in airport_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
airport_venues = pd.DataFrame(documents)
airport_venues["place"] = "airport"
airport_venues.head()

,name,location,place
0,JFK Terminal 7 British Airways,"{'type': 'Point', 'coordinates': [-73.90984078...",airport
1,Pratt Car Service,"{'type': 'Point', 'coordinates': [-73.95354451...",airport
2,Brooklyn Car Service Newark Airport,"{'type': 'Point', 'coordinates': [-73.95225763...",airport
3,Quality Car & Limousine Service,"{'type': 'Point', 'coordinates': [-73.929799, ...",airport
4,Silvercup Limousine Inc,"{'type': 'Point', 'coordinates': [-73.92276, 4...",airport


In [33]:
airport_venues.to_json("airport_venues_ny.json", orient="records")

In [34]:
parametros_5 = {"client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "pet hairdresser", 
    "limit": 100    
}

In [35]:
resp_5 = requests.get(url= url_query, params = parametros_5).json()
data_5 = resp_5.get("response").get("groups")[0].get("items")
data_5

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e47393d1850c7ee212daa39',
   'name': "Dee's Pet Food",
   'contact': {},
   'location': {'address': '716 Classon Ave',
    'crossStreet': 'Btwn. Prospect Place & Park Place',
    'lat': 40.675601,
    'lng': -73.959982,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.675601,
      'lng': -73.959982},
     {'label': 'entrance', 'lat': 40.675563, 'lng': -73.95996}],
    'distance': 2207,
    'postalCode': '11238',
    'cc': 'US',
    'city': 'Brooklyn',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['716 Classon Ave (Btwn. Prospect Place & Park Place)',
     'Brooklyn, NY 11238',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d100951735',
     'name': 'Pet Store',
     'pluralName': 'Pet Stores',
     'shortName': 'Pet Store',
     'icon': {'prefix': 'https://ss3.4s

In [36]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

dog_list = []
for dic in data_5:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    dog_list.append(paralista)
    

documents = []
for diccionario in dog_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
dog_venues = pd.DataFrame(documents)
dog_venues["place"] = "dog"
dog_venues.head()

,name,location,place
0,Dee's Pet Food,"{'type': 'Point', 'coordinates': [-73.959982, ...",dog
1,I Love My Pet Food & Supplies,"{'type': 'Point', 'coordinates': [-73.950556, ...",dog
2,Pet Haven Animal Hospital,"{'type': 'Point', 'coordinates': [-73.98031536...",dog
3,Acme Pet Food Inc.,"{'type': 'Point', 'coordinates': [-73.96900612...",dog
4,Pet boutique & Supplies,"{'type': 'Point', 'coordinates': [-73.98309726...",dog


In [37]:
dog_venues.to_json("dog_venues_ny.json", orient="records")

In [38]:
parametros_6 = {"client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{ny.get('coordinates')[0]},{ny.get('coordinates')[1]}",
    "query": "club", 
    "limit": 100    
}

In [39]:
resp_6 = requests.get(url= url_query, params = parametros_6).json()
data_6 = resp_6.get("response").get("groups")[0].get("items")
data_6

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5a35f6ae23472452931acac0',
   'name': "Basquiat's Bottle",
   'contact': {},
   'location': {'address': '1198 Fulton St',
    'lat': 40.68052683711768,
    'lng': -73.95263170515346,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.68052683711768,
      'lng': -73.95263170515346}],
    'distance': 2164,
    'postalCode': '11216',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['1198 Fulton St',
     'New York, NY 11216',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d11f941735',
     'name': 'Nightclub',
     'pluralName': 'Nightclubs',
     'shortName': 'Nightclub',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   '

In [40]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

club_list = []
for dic in data_6:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    club_list.append(paralista)
    

documents = []
for diccionario in club_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
club_venues = pd.DataFrame(documents)
club_venues["place"] = "club"
club_venues.head()

,name,location,place
0,Basquiat's Bottle,"{'type': 'Point', 'coordinates': [-73.95263170...",club
1,The Buzz Nightclub,"{'type': 'Point', 'coordinates': [-73.95863448...",club
2,Friends and Lovers,"{'type': 'Point', 'coordinates': [-73.95871240...",club
3,KAI studio,"{'type': 'Point', 'coordinates': [-73.95669545...",club
4,Pacific BK,"{'type': 'Point', 'coordinates': [-73.957894, ...",club


In [41]:
club_venues.to_json("club_venues_ny.json", orient="records")

In [51]:
ny_venues = db.get_collection("ny_venues")

In [85]:
ny_coordinates = [-73.986955, 40.745154] 
coord_point = {"type":"Point", "coordinates": ny_coordinates}

In [97]:
query = {"location": {"$near": {"$geometry": coord_point,"$minDistance": 0  , "$maxDistance": 1500}}}

In [98]:
query_final = list(ny_venues.find(query))

In [99]:
ny_final = pd.DataFrame(query_final)
ny_final

,_id,name,location,place
0,607dbd2a202adb1ce64cef03,Next Jump,"{'type': 'Point', 'coordinates': [-73.9869549,...",company
1,607dbd2a202adb1ce64cefb6,StyleCaster,"{'type': 'Point', 'coordinates': [-73.9900984,...",company
2,607dbd2a202adb1ce64cee2d,MediaTrust,"{'type': 'Point', 'coordinates': [-73.984268, ...",company
3,607dbd2a202adb1ce64cee41,Next New Networks,"{'type': 'Point', 'coordinates': [-73.983626, ...",company
4,607dbd2a202adb1ce64cef36,[x+1],"{'type': 'Point', 'coordinates': [-73.982624, ...",company
5,607dbd2a202adb1ce64cee84,[x+1],"{'type': 'Point', 'coordinates': [-73.982624, ...",company
6,607dbd2a202adb1ce64ceeca,Conductor,"{'type': 'Point', 'coordinates': [-73.982596, ...",company
7,607dbd2a202adb1ce64cee0d,Yodle,"{'type': 'Point', 'coordinates': [-73.9918126,...",company
8,607dbd2a202adb1ce64cedf0,Clickable,"{'type': 'Point', 'coordinates': [-73.9912208,...",company
9,607dbd2a202adb1ce64cedd8,SelectMinds,"{'type': 'Point', 'coordinates': [-73.990004, ...",company


In [100]:
latitude = [lat["location"]["coordinates"][1] for lat in query_final]
longitude = [long["location"]["coordinates"][0] for long in query_final]

#De aquí añadimos las listas que nos interesen al DataFrame de la ciudad de San Francisco.
ny_final["latitude"] = latitude
ny_final["longitude"] = longitude


ny_final = ny_final[ny_final["latitude"].notna()]
ny_final = ny_final[ny_final["longitude"].notna()]

ny_final_companies = ny_final[ny_final["place"] == "company"]
ny_final_companies

,_id,name,location,place,latitude,longitude
0,607dbd2a202adb1ce64cef03,Next Jump,"{'type': 'Point', 'coordinates': [-73.9869549,...",company,40.745154,-73.986955
1,607dbd2a202adb1ce64cefb6,StyleCaster,"{'type': 'Point', 'coordinates': [-73.9900984,...",company,40.745050,-73.990098
2,607dbd2a202adb1ce64cee2d,MediaTrust,"{'type': 'Point', 'coordinates': [-73.984268, ...",company,40.743662,-73.984268
3,607dbd2a202adb1ce64cee41,Next New Networks,"{'type': 'Point', 'coordinates': [-73.983626, ...",company,40.743808,-73.983626
4,607dbd2a202adb1ce64cef36,[x+1],"{'type': 'Point', 'coordinates': [-73.982624, ...",company,40.745507,-73.982624
5,607dbd2a202adb1ce64cee84,[x+1],"{'type': 'Point', 'coordinates': [-73.982624, ...",company,40.745507,-73.982624
6,607dbd2a202adb1ce64ceeca,Conductor,"{'type': 'Point', 'coordinates': [-73.982596, ...",company,40.746332,-73.982596
7,607dbd2a202adb1ce64cee0d,Yodle,"{'type': 'Point', 'coordinates': [-73.9918126,...",company,40.742459,-73.991813
8,607dbd2a202adb1ce64cedf0,Clickable,"{'type': 'Point', 'coordinates': [-73.9912208,...",company,40.741356,-73.991221
9,607dbd2a202adb1ce64cedd8,SelectMinds,"{'type': 'Point', 'coordinates': [-73.990004, ...",company,40.740251,-73.990004


In [101]:
ny_map = folium.Map(location = [40.745154, -73.986955], zoom_start = 15)
for i, row in ny_final_companies.iterrows():
    distrito = {
        "location":[row["latitude"], row["longitude"]],
        "tooltip": [row["name"], "aquí tienes tus putos sitios, Pablo"]}
    
    icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "map-pin",
                    icon_color = "black")
        
        
    
    Marker(**distrito,icon = icon ).add_to(ny_map)
ny_map      